Updated notebook - main changes: revert to separating each category into its own collection, implement 2 step pipeline (categorize, then retrieve from those relevant collections). Somehow, takes a substantial amount of time now (ie 7 queries in 17 minutes) (using general qa set and some other random set of data), to discuss.

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
!pip install llama-cpp-python

Define Variables

In [3]:
from llama_cpp import Llama
from langchain_community.llms import HuggingFaceEndpoint
import os
from transformers import pipeline

model_path = "mistral-7b-instruct-v0.2.Q4_0.gguf"
model = Llama(model_path=model_path, n_ctx=2048, n_threads=8, verbose=False)

# repo_id = "google/gemma-2-2b-it"

# hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# # obv params, max_length is max token len for generated text, temp=0.1 means give more predictable and less random results
# model = HuggingFaceEndpoint(
#     task='text-generation',
#     repo_id=repo_id,
#     max_length=1024,
#     temperature=0.1,
#     huggingfacehub_api_token=hf_token
# )

/opt/anaconda3/envs/CampusConnect/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kern

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb

# pt model for generating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")


/var/folders/fz/m3b664vj5hq4kdmwr645k9_h0000gn/T/ipykernel_79363/3407937140.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Define Data Sources

In [5]:
import pandas as pd
import concurrent.futures
import uuid
import os

file_names = [
    "study_permit_general", "work_permit_student_general", "work-study-data-llm",
    "vancouver_transit_qa_pairs", "permanent_residence_student_general", "data-with-sources",
    "faq_qa_pairs_general", "hikes_qa", "sfu-faq-with-sources", "sfu-housing-with-sources",
    "sfu-immigration-faq", "park_qa_pairs-up", "cultural_space_qa_pairs_up",
    "qa_pairs_food", "qa_pairs_year_and_month_avg", "qa_pairs_sfu_clubs"
]

collections = {}
batch_size = 32

def process_file(file):
    try:
        path = f'../Data/{file}.csv'
        if not os.path.exists(path):
            return f"{file} skipped (file not found)."

        df = pd.read_csv(path, usecols=lambda col: col.lower() in {"question", "answer"})
        df.columns = df.columns.str.lower()

        if "question" not in df.columns or "answer" not in df.columns:
            return f"{file} skipped (missing question/answer columns)."

        df = df.drop_duplicates(subset="question")
        df["text"] = df["question"].fillna('') + ' ' + df["answer"].fillna('')
        unique_texts = list(set(df["text"].dropna().tolist()))

        collection = client.get_or_create_collection(name=file)
        for i in range(0, len(unique_texts), batch_size):
            batch = unique_texts[i:i + batch_size]
            embeddings = embedding_model.embed_documents(batch)
            ids = [str(uuid.uuid4()) for _ in batch]
            collection.add(ids=ids, embeddings=embeddings, documents=batch)

        collections[file] = collection
        return f"{file}: Loaded {len(unique_texts)} docs."
    except Exception as e:
        return f"{file}: Error - {e}"

# parallelogram
with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    results = list(executor.map(process_file, file_names))

for result in results:
    print(result)


study_permit_general: Loaded 14 docs.
work_permit_student_general: Loaded 23 docs.
work-study-data-llm: Loaded 178 docs.
vancouver_transit_qa_pairs: Loaded 37 docs.
permanent_residence_student_general: Loaded 6 docs.
data-with-sources: Loaded 75 docs.
faq_qa_pairs_general: Loaded 220 docs.
hikes_qa: Loaded 278 docs.
sfu-faq-with-sources: Loaded 102 docs.
sfu-housing-with-sources: Loaded 74 docs.
sfu-immigration-faq: Loaded 83 docs.
park_qa_pairs-up: Loaded 216 docs.
cultural_space_qa_pairs_up: Loaded 560 docs.
qa_pairs_food: Loaded 38 docs.
qa_pairs_year_and_month_avg: Loaded 1488 docs.
qa_pairs_sfu_clubs: Loaded 163 docs.


In [6]:
# define cat to collection mapping
# motivation: takes wayyyy too long now -> dataset size trippled and time grew exponentially...
# now takes around 2 mins on avg for each response..compared to 40 seconds previously..

collection_map = {
    "study": "study_permit_general",
    "student work": "work_permit_student_general",
    "work-study": "work-study-data-llm",
    "transit": "vancouver_transit_qa_pairs",
    "permanent residence": "permanent_residence_student_general",
    "general info": "data-with-sources",
    "faq": "faq_qa_pairs_general",
    "hiking": "hikes_qa",
    "sfu faq": "sfu-faq-with-sources",
    "housing": "sfu-housing-with-sources",
    "immigration": "sfu-immigration-faq",
    "parks": "park_qa_pairs-up",
    "culture": "cultural_space_qa_pairs_up",
    "food": "qa_pairs_food",
    "weather": "qa_pairs_year_and_month_avg",
    "clubs": "qa_pairs_sfu_clubs"
}


Function to now match for releveant document

In [7]:
def get_relevant_documents(query, categories, n_results=3):
    all_results = []
    query_embedding = embedding_model.embed_documents([query])[0]

    for category in categories:
        collection_name = collection_map[category]
        if collection_name in collections:
            try:
                result = collections[collection_name].query(
                    query_embeddings=[query_embedding],
                    n_results=n_results
                )
                docs = result.get("documents", [[]])[0]
                sims = result.get("distances", [[]])[0]

                all_results.extend(zip(docs, sims))
            except Exception as e:
                print(f"error querying {collection_name}: {e}")

    all_results = sorted(all_results, key=lambda x: x[1])

    return all_results[:n_results]


### Classify Prompt

In [8]:
import re
import difflib

valid_categories = list(collection_map.keys())
fallback_category = "faq"

def classify_query(query):
    category_prompt = f"""
    You are a classifier for a Q&A system for international students in British Columbia.
    Choose the **1 most relevant** category from this list, or at most 3 if absolutely needed (comma-separated):

    {", ".join(valid_categories)}

    Query: "{query}"

    Return only the category name(s) as a comma-separated string.
    """

    response = model(category_prompt, max_tokens=50, temperature=0.1)["choices"][0]["text"].strip().lower()
    print("Raw out:", response)

    tokens = re.findall(r'\b\w+\b', response)

    matched = []
    for token in tokens:
        closest = difflib.get_close_matches(token, valid_categories, n=1, cutoff=0.8)
        if closest and closest[0] not in matched:
            matched.append(closest[0])
        if len(matched) == 3:
            break

    if fallback_category not in matched:
      matched.append(fallback_category)

    return matched[:3]


Generate Answer

In [9]:
def generate_answer(query):
    categories = classify_query(query)
    print(f"Categories {categories}\n")
    relevant_documents = get_relevant_documents(query, categories)

    if not relevant_documents:
        return {
            "Response": "Sorry, no relevant documents found."
        }

    #relevant_documents = list(set(relevant_documents))

    seen = set()
    unique_docs = []
    for doc, sim in relevant_documents:
        if doc not in seen:
            seen.add(doc)
            unique_docs.append((doc, sim))

    print("Relevant Documents with Similarity Scores:")
    for doc, sim in unique_docs:
        print(f"Similarity: {sim:.4f}\nDoc: {doc}\n")

    relevant_texts = "\n\n".join([doc for doc, _ in unique_docs])

    rag_prompt = f"""
    You are a helpful, friendly assistant for international students new to British Columbia, Canada.

    Below are some reference documents that may be relevant to the user's question:
    {relevant_texts}

    INSTRUCTIONS:
    1. If the user's query is just a greeting (like "hello", "hi", "what's up"):
       - Respond with a single brief friendly greeting
       - Offer to help with questions about studying or living in BC
       - Do NOT include ANY information from the reference documents
       - Do NOT create additional questions and answers

    2. If the user is asking for information:
       - Be friendly and answer based ONLY on the reference documents if relevant
       - Keep your answer brief and concise (under 30 words when possible)
       - If the documents don't provide sufficient information, say "I don't have enough information to answer that. Please refer to official sources."
       - Do NOT create additional questions and answers beyond answering their original question

    3. IMPORTANT: Never generate additional content beyond answering the user's question

    User question: {query}

    Your response (just the answer, no preamble):
    """

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]
    # response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)

    return {
        "Response": response_after_rag
    }


Example Usage

In [10]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

In [30]:
answer = generate_answer("What is the upass BC for students in vancouver transit?")
print(answer["Response"])

Raw out: answer: transit
Categories ['transit', 'faq']

Relevant Documents with Similarity Scores:
Similarity: 27.6120
Doc: How do I pay for the bus in Vancouver? In Vancouver, you pay for public transit using a Compass Card, which you can purchase at SkyTrain stations, London Drugs stores, and other locations. As an international student, you're likely eligible for the discounted U-Pass BC program through your institution.

Similarity: 28.5323
Doc: Do international students get discounts on Vancouver public transit? Yes, international students at recognized post-secondary institutions can get a discounted Compass Card (U-Pass BC) which provides unlimited transit use. This is typically included in your student fees.

Similarity: 28.6289
Doc: What is a U-Pass and how do I get one as an international student? A U-Pass BC is a discounted transit pass available to eligible students at participating post-secondary institutions in Metro Vancouver. As an international student, you typically r

In [11]:
import pandas as pd

benchmark_data = pd.read_csv("../poorvi-test-QuesAns.csv")

for idx, row in benchmark_data.iterrows():
    user_query = row["Question"]
    correct_answer = row["Answer"]

    responses = generate_answer(user_query)

    print("\n" + "="*50)
    print(f"Benchmark Query {idx + 1}: {user_query}")
    print("="*50)
    print("\nRAG Response:\n", responses.get("Response", "N/A"))
    print("\n(Benchmark) Answer:\n", correct_answer)
    print("="*50 + "\n\n")


Raw out: ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Categories ['faq']

Relevant Documents with Similarity Scores:
Similarity: 38.0033
Doc: What type of public transit is available in BC? Most cities and towns in British Columbia have a public transit system. BC Transit is 